In [1]:
import os, sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from organelles_classes import cell, nucleus, intensities
import functions as fct
import re

In [10]:
lbdas = [405, 488, 561, 638, 730] # TO DO : collect from iteration across folder instead

# file preparation
data_folder = "data/images"
mask_folder = "data/Masks"
output_file = "data/median_morphometry_per_site.csv"
if os.path.exists(output_file):
    print("Existing outputfile found. Deleting it.")
    os.remove(output_file)
with open(output_file, "w") as file:  
    file.write("well_id,site_id,cell_tot,area,min_diameter,max_diameter,intensities\n")

# harvesting the images        
for index, mask_file in enumerate(os.listdir(mask_folder)):
    if os.path.isdir(os.path.join(mask_folder,mask_file)):
        continue
    mask_file_strs = re.split("[_.]",  mask_file) #(mask_file.split("_")).split(".")
    well_id = mask_file_strs[1]
    site_id = "_".join(mask_file_strs[2:5])
    cell_lists = []
    print("working with set corresponding to ",mask_file)
    im_stack =  cv2.imread(os.path.join(mask_folder, mask_file), cv2.IMREAD_UNCHANGED)
    for l in lbdas:
        filename = f"{well_id}_{site_id}_Fluorescence_{l}_nm_Ex.tiff"
        filepath = os.path.join(data_folder, filename)
        if not os.path.exists(filepath):
            print("missing image", filepath)
            sys.exit()
        im = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        im_stack = np.dstack((im_stack, im))
    im_stack = np.moveaxis(im_stack, -1, 0)
    nb_cell = np.max(im_stack[0])
    cells = []
    if nb_cell <= 0:
        continue
    for i in range(1, nb_cell+1):     # TO DO : PARALLELISATION WITH import multiprocessing as mp pool.map
        index_list = np.where(im_stack[0] == i)
        my_cell = cell.Cell(i, index_list)
        my_cell.calculate_morphometrics() 
        # TO DO : ELIMINATE CELL TOUCHING IMAGE BORDERS      
        #my_cell.calculate_intensity_metrics(im_stack[1:]) 
        # TO DO : function to measure cell distribution within image
        cells.append(my_cell)
    cell_lists.append(cells)
    print("img set",index, len(cells), " cell morpho saved in", output_file )
    fct.save_median_morphometry(cells, site_id[:2], well_id, output_file)

Existing outputfile found. Deleting it.
working with set corresponding to  MASK_B02_s1_x0_y0.tiff
img set 0 125  cell morpho saved in data/median_morphometry_per_site.csv
working with set corresponding to  MASK_B02_s2_x1_y0.tiff
img set 1 100  cell morpho saved in data/median_morphometry_per_site.csv
working with set corresponding to  MASK_B03_s1_x0_y0.tiff
img set 2 48  cell morpho saved in data/median_morphometry_per_site.csv
working with set corresponding to  MASK_B03_s2_x1_y0.tiff
img set 3 49  cell morpho saved in data/median_morphometry_per_site.csv
working with set corresponding to  MASK_B04_s1_x0_y0.tiff
img set 4 202  cell morpho saved in data/median_morphometry_per_site.csv
working with set corresponding to  MASK_B04_s2_x1_y0.tiff
img set 5 231  cell morpho saved in data/median_morphometry_per_site.csv
working with set corresponding to  MASK_B05_s1_x0_y0.tiff
img set 6 231  cell morpho saved in data/median_morphometry_per_site.csv
working with set corresponding to  MASK_B05_s

In [7]:
# for debugging    #test = np.zeros((2500, 2500))
#test[np.where(im_stack[0] == i)] = 1  
#plt.imshow(test, cmap ="gray")
#site_ids = [line.strip() for line in open("data/site_ids.txt", "r")]
#well_ids = [line.strip() for line in open("data/well_ids.txt", "r")]

In [ ]:
print(my_cell.morphometry)

In [ ]:
im_stack_reordered.shape